In [15]:
import pandas as pd
from sqlalchemy import create_engine
from config import username, password

In [16]:
#import happiness data to a DF
csv_file = "Resources/Happydatasets_748584_1294786_2015.csv"
happy_df = pd.read_csv(csv_file)
happy_df = happy_df.rename(columns={"Country":'country', "Happiness Rank":"happiness_rank","Health (Life Expectancy)":"life_expectancy", "Freedom":"freedom", })
happy_df = happy_df[["country", "happiness_rank", "life_expectancy", "freedom"]]
happy_df = happy_df.set_index('country')
happy_df.head()

,happiness_rank,life_expectancy,freedom
country,,,
Switzerland,1,0.94143,0.66557
Iceland,2,0.94784,0.62877
Denmark,3,0.87464,0.64938
Norway,4,0.88521,0.66973
Canada,5,0.90563,0.63297


In [17]:
#import drinking data to a DF
csv_file = "Resources/datasets_105709_251851_drinks.csv"
drinking_df = pd.read_csv(csv_file)
drinking_df = drinking_df.set_index('country')
drinking_df = drinking_df[["beer_servings","spirit_servings", "wine_servings","total_litres_of_pure_alcohol"  ]]
drinking_df.head()

,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol
country,,,,
Afghanistan,0,0,0,0.0
Albania,89,132,54,4.9
Algeria,25,0,14,0.7
Andorra,245,138,312,12.4
Angola,217,57,45,5.9


In [18]:
#import GDP data to a DF
csv_file = "Resources/GDPdatasets_412229_789160_Dataset.csv"
gdp_df = pd.read_csv(csv_file)
gdp_df = gdp_df[["CountryName","GDP_PerCapita_2015 US$", "SuicideRate_2015" ]]
gdp_df = gdp_df.rename(columns={"CountryName":'country', "GDP_PerCapita_2015 US$":"gdp_pc_usdollar","SuicideRate_2015":"suicide_rate"})
gdp_df = gdp_df.set_index("country")
gdp_df.head()

,gdp_pc_usdollar,suicide_rate
country,,
Afghanistan,578.466353,4.8
Angola,4166.979684,5.0
Albania,3952.830781,6.0
United Arab Emirates,38663.383810,2.8
Argentina,13789.060420,8.8


In [19]:
#import obesity data to a DF
csv_file = "Resources/WHOdata.csv"
obesity_df = pd.read_csv(csv_file)
obesity_df = obesity_df.drop(obesity_df.index[[0,1]])
obesity_df.columns = obesity_df.iloc[0]
obesity_df = obesity_df.drop(obesity_df.index[0])
obesity_df = obesity_df.rename(columns={"Country":"country","Both sexes": "obesity_pct"})
obesity_df = obesity_df.set_index("country")
obesity_df["obesity_pct"] = obesity_df["obesity_pct"].str.split(" ").str[0]
obesity_df = obesity_df[obesity_df.obesity_pct != 'No']
obesity_df.to_csv("obesity_data.csv")
obesity_df

2,obesity_pct
country,
Afghanistan,22.3
Albania,57.0
Algeria,61.3
Andorra,63.5
Angola,26.8
...,...
Venezuela (Bolivarian Republic of),63.0
Viet Nam,17.7
Yemen,48.0


In [20]:
#join gdp and happiness dataframes
happy_df = pd.merge(happy_df, gdp_df, on='country')
happy_df.head()

,happiness_rank,life_expectancy,freedom,gdp_pc_usdollar,suicide_rate
country,,,,,
Switzerland,1,0.94143,0.66557,82081.59712,17.1
Iceland,2,0.94784,0.62877,52428.59576,14.0
Denmark,3,0.87464,0.64938,53254.85600,11.9
Norway,4,0.88521,0.66973,74521.56993,12.4
Canada,5,0.90563,0.63297,43495.05439,12.5


In [21]:
rds_connection_string = f"{username}:{password}@localhost:5432/ETL_Proj"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [25]:
happy_df.to_sql(name='country_happiness_index', con=engine, if_exists='append', index=True)

IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "country_happiness_index_pkey"
DETAIL:  Key (country)=(Switzerland) already exists.

[SQL: INSERT INTO country_happiness_index (country, happiness_rank, life_expectancy, freedom, gdp_pc_usdollar, suicide_rate) VALUES (%(country)s, %(happiness_rank)s, %(life_expectancy)s, %(freedom)s, %(gdp_pc_usdollar)s, %(suicide_rate)s)]
[parameters: ({'country': 'Switzerland', 'happiness_rank': 1, 'life_expectancy': 0.9414299999999999, 'freedom': 0.66557, 'gdp_pc_usdollar': 82081.59712, 'suicide_rate': 17.1}, {'country': 'Iceland', 'happiness_rank': 2, 'life_expectancy': 0.94784, 'freedom': 0.6287699999999999, 'gdp_pc_usdollar': 52428.595760000004, 'suicide_rate': 14.0}, {'country': 'Denmark', 'happiness_rank': 3, 'life_expectancy': 0.87464, 'freedom': 0.6493800000000001, 'gdp_pc_usdollar': 53254.856, 'suicide_rate': 11.9}, {'country': 'Norway', 'happiness_rank': 4, 'life_expectancy': 0.88521, 'freedom':

In [26]:
obesity_df.to_sql(name='country_obesity', con=engine, if_exists='append', index=True)

In [27]:
drinking_df.to_sql(name='country_alcohol_consumption', con=engine, if_exists='append', index=True)